In [1]:
import GLPE
#https://github.com/ekehoe32/orthrus
import sys
sys.path.append('/home/katrina/a/mankovic/ZOETIS/Fall2021/Orthrus/orthrus')
import orthrus
from orthrus import core
from orthrus.core import dataset
import numpy as np
# from NetworkDataAnalysis import graph_tools_construction as gt
from matplotlib import pyplot as plt
import pandas
# from orthrus.core.pipeline import *
from sklearn.preprocessing import FunctionTransformer
from orthrus.preprocessing.imputation import HalfMinimum
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from orthrus.core.helper import load_object
import os

In [2]:
#load the dataset
ds = pandas.read_csv('/data4/mankovic/GSE73072/calcom_splits/data/gse73072_5to1_25_32_limma_train.csv', index_col = 'SampleID')

In [3]:
#maybe do some dataset normalization here?

# Simple LPE with GSE data

In [4]:
#init lpe
lpe = GLPE.LPE( feature_ids = list(ds.columns), #list of the probeids or entrez ids for the dataset 
                pathway_files = '/data4/mankovic/GSE73072/new_pw_edge_mtx/', #probeid networks
                normalize_rows=True ) # normalize rows of pathway expression matrix)

#fit to make pathway transition matrix
lpe.fit(X = np.array(ds))

#transform data
pathway_expression_matrix = lpe.transform(np.array(ds))



/home/katrina/a/mankovic/PathwayAnalysis/GLPE.py:191: RuntimeWarning: invalid value encountered in true_divide
  row = row/np.sum(row)


In [5]:
#combine the results into a pathway expression matrix dataframe
pathway_expression_df = pandas.DataFrame(data = pathway_expression_matrix, columns = lpe.pathway_names_, index = ds.index)
pathway_expression_df

,R-HSA-109582,R-HSA-1430728,R-HSA-1266738,R-HSA-109581,R-HSA-212436,R-HSA-168249,R-HSA-162582,R-HSA-397014,R-HSA-1500931,R-HSA-166016,...,R-HSA-1368071,R-HSA-170145,R-HSA-168270,R-HSA-168288,R-HSA-168298,R-HSA-168302,R-HSA-168303,R-HSA-168336,R-HSA-168874,R-HSA-192869
SampleID,,,,,,,,,,,,,,,,,,,,,
GSM1881744,7.555032,7.508696,7.454280,8.168684,7.249438,8.473554,7.100261,6.460005,7.044749,8.194195,...,8.493806,4.552984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GSM1881745,7.558103,7.527427,7.481479,8.191609,7.281804,8.463511,7.115830,6.454360,7.086558,8.172271,...,8.582773,4.675716,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GSM1881749,7.561875,7.519266,7.451775,8.211136,7.281491,8.482233,7.105678,6.432204,7.047700,8.215761,...,8.670156,4.649953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GSM1881764,7.545506,7.501286,7.445481,8.231406,7.264536,8.566684,7.123671,6.434900,7.094688,8.273793,...,8.633616,4.833279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GSM1881765,7.536281,7.508743,7.428549,8.261101,7.271526,8.580936,7.100765,6.397225,7.029696,8.288920,...,8.514880,5.100471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1884181,7.561046,7.469872,7.423996,8.109689,7.256253,8.458397,7.122629,6.388764,7.144233,8.174429,...,8.794682,4.479391,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GSM1884182,7.551168,7.461632,7.426054,8.045137,7.236001,8.414756,7.116041,6.419725,7.156535,8.138180,...,8.666497,4.432585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GSM1884195,7.603101,7.460094,7.428315,8.028234,7.239460,8.414418,7.113060,6.463737,7.158515,8.134022,...,8.720217,4.804831,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# GSE Test

In [6]:
#generate clpe
precomputed_clpe = GLPE.CLPE(centrality_measure = 'page_rank', #choose 'page_rank' or 'degree'
                    network_type = 'precomputed', #choose 'precomputed', 'correlation' or 'heatkernel'
                    feature_ids = list(ds.columns), #list of the probeids or entrez ids for the dataset 
                    pathway_files = '/data4/mankovic/GSE73072/new_pw_edge_mtx/', #probeid networks
                    # pathway_files = '/data3/darpa/omics_databases/ensembl2pathway/pathways_edges/pathways/pw_edge_mtx/', #entrezid netowrks
                    directed = True, # directed or undirected network (useful for precomputed)
                    heat_kernel_param=2, # parameter for heatkernel edges
                    normalize_rows=True ) # normalize rows of pathway expression matrix

#fit clpe to generate pathway expression matrix
precomputed_clpe.fit(np.array(ds))

/home/katrina/a/mankovic/PathwayAnalysis/GLPE.py:450: RuntimeWarning: invalid value encountered in true_divide
  scores = scores/np.sum(scores)


CLPE(centrality_measure='page_rank', directed=True,
     feature_ids=['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at',
                  '1294_at', '1316_at', '1320_at', '1405_i_at', '1431_at',
                  '1438_at', '1487_at', '1494_f_at', '1598_g_at', '160020_at',
                  '1729_at', '177_at', '1773_at', '179_at', '1861_at',
                  '200000_s_at', '200001_at', '200002_at', '200003_s_at',
                  '200004_at', '200005_at', '200006_at', '200007_at',
                  '200008_s_at', '200009_at', ...],
     heat_kernel_param=2.0, network_type='precomputed',
     pathway_files='/data4/mankovic/GSE73072/new_pw_edge_mtx/',
     pathway_transition_matrix=<2261x22277 sparse matrix of type '<class 'numpy.float64'>'
	with 169541 stored elements in Compressed Sparse Row format>)

In [7]:
#transform data to make pathway expression matrix
clpe_pathway_expression_matrix = precomputed_clpe.transform(np.array(ds))

In [8]:
#combine the results into a pathway expression matrix dataframe
clpe_pathway_expression_df = pandas.DataFrame(data = clpe_pathway_expression_matrix, columns = precomputed_clpe.pathway_names_, index = ds.index)
clpe_pathway_expression_df

,R-HSA-109582,R-HSA-1430728,R-HSA-1266738,R-HSA-109581,R-HSA-212436,R-HSA-168249,R-HSA-162582,R-HSA-397014,R-HSA-1500931,R-HSA-166016,...,R-HSA-1368071,R-HSA-170145,R-HSA-168270,R-HSA-168288,R-HSA-168298,R-HSA-168302,R-HSA-168303,R-HSA-168336,R-HSA-168874,R-HSA-192869
SampleID,,,,,,,,,,,,,,,,,,,,,
GSM1881744,7.609675,8.236543,7.738651,8.466643,7.595100,8.755626,7.530879,6.559509,7.072858,8.133057,...,8.690978,4.552802,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GSM1881745,7.610222,8.268846,7.759241,8.477739,7.610757,8.746821,7.535806,6.537450,7.102214,8.137394,...,8.828375,4.675539,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GSM1881749,7.628206,8.260702,7.744352,8.520574,7.623532,8.781187,7.547209,6.522398,7.084354,8.207203,...,8.816468,4.649640,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GSM1881764,7.618515,8.191531,7.717402,8.517461,7.601273,8.812603,7.555688,6.514761,7.130020,8.206196,...,8.746222,4.832982,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GSM1881765,7.607285,8.226577,7.707110,8.566879,7.614125,8.829007,7.543260,6.461545,7.065540,8.238575,...,8.629156,5.099986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1884181,7.628345,8.125992,7.697965,8.358097,7.590428,8.732213,7.533870,6.448691,7.169438,8.142851,...,8.924529,4.479107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GSM1884182,7.610878,8.072785,7.681911,8.301618,7.561833,8.680354,7.515943,6.483052,7.180878,8.048436,...,8.893244,4.432299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GSM1884195,7.674807,8.092200,7.701681,8.287281,7.598827,8.667395,7.531297,6.550226,7.177825,8.047474,...,8.867960,4.804286,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
